In [ ]:
import os
from IPython.display import IFrame
import pandas as pd
import sys
sys.path.insert(0, '../JKBio/')
import epigenetics.ChIP_helper as chiphelper
import Helper as helper
import igv
import numpy as np
import pyBigWig
import itertools
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from bokeh.plotting import *
from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
output_notebook()
%load_ext autoreload
%autoreload 2

# ChIP

In [ ]:
!gsutil ls gs://amlproject/Chip_AML/Chip/

In [ ]:
a = !gsutil ls gs://amlproject/Chip_AML/Chip/IRF2BP2/

In [ ]:
rename1 = {
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_10_MP7613_S8_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp910_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_10_MP7613_S8_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp910_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_11_MP7613_S9_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp911_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_11_MP7613_S9_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp911_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_12_MP7613_S10_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp912_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_12_MP7613_S10_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp912_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_13_MP7613_S11_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp913_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_13_MP7613_S11_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp913_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_14_MP7613_S12_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp914_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_14_MP7613_S12_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp914_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_15_MP7613_S13_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp915_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_15_MP7613_S13_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp915_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_16_MP7613_S14_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp916_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_16_MP7613_S14_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp916_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_1_MP7613_S1_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp901_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_1_MP7613_S1_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp901_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_2_MP7613_S2_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp902_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_2_MP7613_S2_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp902_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_3_MP7613_S3_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp903_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_3_MP7613_S3_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp903_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_4_MP7613_S4_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp904_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_4_MP7613_S4_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp904_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_5R_MP7613_S15_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp905_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_5R_MP7613_S15_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp905_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_6R_MP7613_S16_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp906_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_6R_MP7613_S16_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp906_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_7_MP7613_S5_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp907_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_7_MP7613_S5_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp907_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_8_MP7613_S6_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp908_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_8_MP7613_S6_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp908_2.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_9_MP7613_S7_R1_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp909_1.fastq.gz",
 'gs://amlproject/Chip_AML/Chip/IRF2BP2/20191211_9_MP7613_S7_R2_001.fastq.gz':"gs://amlproject/Chip_AML/Chip/IRF2BP2/mp909_2.fastq.gz"}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
a

In [ ]:
a.append("gs://amlproject/Chip_AML/Chip/mp99.fastq.gz")

In [ ]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')

In [ ]:
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [ ]:
gsheet

In [ ]:
df = {
"name": [],
"fastq1": [],
"fastq2": [],
"quality": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in helper.grouped(a,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('_')[0]]
    df['name'].append(row.id.values[0] + '_'+ row.name_replicate.values[0] +"_"+row.protein.values[0])
    df['fastq1'].append(val[0])
    df['fastq2'].append(val[1])
    df['quality'].append(1)
    df['antibody'].append(row.protein.values[0])
    df['group'].append(row.name_replicate.values[0])
    df['replicate'].append(1)
    df['control'].append(np.nan)
df = pd.DataFrame(df)

In [ ]:
row.id.values[0]

In [ ]:
df[df.columns[[5,6,1,2,3,4,7]]].to_csv('design.csv')

## RNA

In [ ]:
a = !gsutil ls gs://amlproject/slamseq/IRF2BP2/

In [ ]:
rename1 = {
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_10_MP7623_S30_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp810_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_11_MP7623_S31_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp811_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_12_MP7623_S32_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp812_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_13_MP7623_S33_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp813_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_14_MP7623_S34_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp814_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_15_MP7623_S35_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp815_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_16_MP7623_S36_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp816_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_1_MP7623_S21_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp801_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_2_MP7623_S22_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp802_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_3_MP7623_S23_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp803_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_4_MP7623_S24_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp804_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_5_MP7623_S25_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp805_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_6_MP7623_S26_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp806_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_7_MP7623_S27_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp807_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_8_MP7623_S28_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp808_2.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R1_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_1.fastq.gz',
'gs://amlproject/slamseq/IRF2BP2/20191212_9_MP7623_S29_R2_001.fastq.gz':'gs://amlproject/slamseq/IRF2BP2/mp809_2.fastq.gz'}

In [ ]:
for k, val in rename1.items():
    os.system('gsutil mv '+k+' '+val)

In [ ]:
a

In [ ]:
import TerraFunction as terra

In [ ]:
terra.list_blobs_with_prefix("amlproject","slamseq/IRF2BP2/")

In [ ]:
terra.uploadFromFolder("amlproject", "slamseq/IRF2BP2/", workspace="broad-firecloud-ccle/hg38_RNAseq", 
                       fformat="fastq12", source="maximP",samplesetname="slamseq_IRF2BP2")